# Сериализация

## Обработка конфигурационных файлов

### json

JSON (JavaScript Object Notation) - простой формат обмена данными, основанный на подмножестве синтаксиса JavaScript. Модуль json позволяет кодировать и декодировать данные в удобном формате.

Некоторые возможности библиотеки **json**

**json.dump**`(obj, fp, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)` - сериализует obj как форматированный JSON поток в fp.

**json.dumps**`(obj, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)` - сериализует obj в строку JSON-формата.

**json.load**`(fp, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)` - десериализует JSON из fp.

**json.loads**`(s, encoding=None, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)` - десериализует s (экземпляр str, содержащий документ JSON) в объект Python.

In [ ]:
import json

# Кодирование основных объектов Python
print(json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}]))
print(json.dumps({"c": 0, "b": 0, "a": 0}, sort_keys=True))

# Компактное кодирование
print(json.dumps([1,2,3,{'4': 5, '6': 7}], separators=(',', ':')))

# Красивый вывод
print(json.dumps({'4': 5, '6': 7}, sort_keys=True, indent=4))

# Декодирование (парсинг) JSON
print(json.loads('["foo", {"bar":["baz", null, 1.0, 2]}]'))
print(json.loads('"\\"foo\\bar"'))

### yaml

YAML (YAML Ain’t Markup Language) - еще один текстовый формат для записи данных.

YAML более приятен для восприятия человеком, чем JSON, поэтому его часто используют для описания сценариев в ПО. Например, в Ansible.

Для работы с YAML в Python используется модуль **pyyaml**. Он не входит в стандартную библиотеку модулей, поэтому его нужно установить:

`pip install pyyaml`

In [ ]:
# Чтение из YAML (файл info.yaml)

import yaml
from pprint import pprint # Модуль pprint позволяет красиво отображать объекты Python

with open('info.yaml') as f:
    templates = yaml.safe_load(f)

pprint(templates) # Использование функции модуля pprint для вывода

In [ ]:
# Запись в YAML

trunk_template = [
    'switchport trunk encapsulation dot1q', 'switchport mode trunk',
    'switchport trunk native vlan 999', 'switchport trunk allowed vlan'
]

access_template = [
    'switchport mode access', 'switchport access vlan',
    'switchport nonegotiate', 'spanning-tree portfast',
    'spanning-tree bpduguard enable'
]

to_yaml = {'trunk': trunk_template, 'access': access_template}

with open('sw_templates.yaml', 'w') as f:
    yaml.dump(to_yaml, f)

with open('sw_templates.yaml') as f:
    print(f.read())

### ini

Как правило, ini-файлы используют для хранения настроек приложения или операционной системы. Библиотека в ядре Python включает в себя модуль, под названием **configparser**, который вы можете использовать для создания и работы с файлами конфигурации.

In [ ]:
import configparser

# Создание конфигурационного файла
config = configparser.ConfigParser()
config.add_section("Settings")
config.set("Settings", "font", "Courier")
config.set("Settings", "font_size", "10")
config.set("Settings", "font_style", "Normal")
config.set("Settings", "font_info",
           "You are using %(font)s at %(font_size)s pt")
    
with open('my_settings.ini', 'w') as config_file:
    config.write(config_file)

# ===Выведем содержимое файла===
with open('my_settings.ini', 'r') as config_file:
    print(config_file.read())



# Чтение конфигурационного файла
config = configparser.ConfigParser()
config.read('my_settings.ini')
    
# Читаем некоторые значения из конфиг. файла.
font = config.get("Settings", "font")
font_size = config.get("Settings", "font_size")

# Меняем значения из конфиг. файла.
config.set("Settings", "font_size", "12")

# Удаляем значение из конфиг. файла.
config.remove_option("Settings", "font_style")
 
# Вносим изменения в конфиг. файл.
with open('my_settings.ini', "w") as config_file:
    config.write(config_file)

# ===Выведем содержимое файла===
with open('my_settings.ini', 'r') as config_file:
    print(config_file.read())

## Консервация объектов

Модуль `pickle` (англ. pickle - консервировать) реализует мощный алгоритм сериализации и десериализации объектов Python. "Pickling" - процесс преобразования объекта Python в поток байтов, а "unpickling" - обратная операция, в результате которой поток байтов преобразуется обратно в Python-объект. Так как поток байтов легко можно записать в файл, модуль `pickle` широко применяется для сохранения и загрузки сложных объектов в Python.

Модуль pickle предоставляет следующие функции для удобства сохранения/загрузки объектов:

- `pickle.dump(obj, file, protocol=None, *, fix_imports=True)`\
Записывает сериализованный объект в файл. Дополнительный аргумент protocol указывает используемый протокол. По умолчанию равен 3 и именно он рекомендован для использования в Python 3 (несмотря на то, что в Python 3.4 добавили протокол версии 4 с некоторыми оптимизациями). В любом случае, записывать и загружать надо с одним и тем же протоколом.

- `pickle.dumps(obj, protocol=None, *, fix_imports=True)`\
Возвращает сериализованный объект. Впоследствии вы его можете использовать как угодно.

- `pickle.load(file, *, fix_imports=True, encoding="ASCII", errors="strict")`\
Загружает объект из файла.

- `pickle.loads(bytes_object, *, fix_imports=True, encoding="ASCII", errors="strict")`\
Загружает объект из потока байт.

Модуль `pickle` также определяет несколько исключений:

`pickle.PickleError`
- `pickle.PicklingError` - случились проблемы с сериализацией объекта.
- `pickle.UnpicklingError` - случились проблемы с десериализацией объекта.

Этих функций вполне достаточно для сохранения и загрузки встроенных типов данных.



In [ ]:
import pickle
data = {
    'a': [1, 2.0, 3, 4+6j],
    'b': ("character string", b"byte string"),
    'c': {None, True, False}
}

with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

with open('data.pickle', 'rb') as f:
    data_new = pickle.load(f)

print(data_new)